In [1]:
import os
import string
import h5py

import matplotlib.pyplot as plt
import time, pickle, pandas

import numpy as np

import keras
from keras.optimizers import rmsprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import backend as K
import glob
import shutil

import scipy.misc as misc
import scipy.ndimage as im

import tensorflow as tf

%matplotlib inline

Using TensorFlow backend.


In [2]:
batch_size = 10

data_root = '../data/combined/'
train_data_dir = data_root + 'train'
validation_data_dir = data_root + 'test'

train_datagen = ImageDataGenerator(
        shear_range=0.25,
        zoom_range=0.25,
        rotation_range=35,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

batch, y = train_generator.next()
img_height = batch.shape[1]
img_width = batch.shape[2]
channels = batch.shape[3]
num_classes = len(y[0])
print img_height, img_width, channels, num_classes

Found 309 images belonging to 3 classes.
Found 81 images belonging to 3 classes.
256 256 1 3


In [3]:
model = keras.models.load_model('../models/sketchnet4_1/latest.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 524288)            0         
__________

In [4]:
# load latest, add more convs
for i in xrange(4):
    model.pop()
    
model.add(Convolution2D(256, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Convolution2D(512, (3, 3), 
                        padding='same', 
                        activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

for i, layer in enumerate(model.layers):
    layer.name = 'layer'+str(i)

In [5]:
opt = rmsprop(lr=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
import utils
reload(utils)

trainer =  utils.ModelTrainer(
                 model,
                 'augmented1',
                 opt,
                 train_generator,
                 validation_generator,
                 train_steps=31,
                 val_steps=8)

In [7]:
trainer.train(20, 0.0001)

Epoch 1/20
31/31 [==============================] - 17s - loss: 3.5828 - acc: 0.3136 - val_loss: 1.0438 - val_acc: 0.5375
Epoch 2/20
31/31 [==============================] - 19s - loss: 1.0747 - acc: 0.4305 - val_loss: 0.9937 - val_acc: 0.4500
Epoch 3/20
31/31 [==============================] - 23s - loss: 0.9808 - acc: 0.4889 - val_loss: 0.8628 - val_acc: 0.7875
Epoch 4/20
31/31 [==============================] - 18s - loss: 0.9892 - acc: 0.5079 - val_loss: 0.8998 - val_acc: 0.7750
Epoch 5/20
31/31 [==============================] - 18s - loss: 0.9323 - acc: 0.5688 - val_loss: 0.8365 - val_acc: 0.6875
Epoch 6/20
31/31 [==============================] - 18s - loss: 0.9510 - acc: 0.5405 - val_loss: 0.7768 - val_acc: 0.7375
Epoch 7/20
31/31 [==============================] - 18s - loss: 0.8983 - acc: 0.5437 - val_loss: 0.7622 - val_acc: 0.7875
Epoch 8/20
31/31 [==============================] - 23s - loss: 0.8715 - acc: 0.5889 - val_loss: 0.6526 - val_acc: 0.8500
Epoch 9/20
31/31 [======